In [15]:
import pandas as pd

# Path to your file
file_path = "/home/ec2-user/SageMaker/InterPLM/oas_data/1a_S1mod2_S1__1_Paired_All.csv.gz"

# Load the first few rows to inspect
df = pd.read_csv(file_path, compression="gzip")

# Show basic info about the dataframe
print("=== Columns ===")
print(df.columns.tolist())

print("\n=== Shape ===")
print(df.shape)

print("\n=== First 5 rows ===")
print(df.head())

# If you want to see more detailed info (types, non-null counts)
print("\n=== Info ===")
df.info()


=== Columns ===
['{"Run": "1a_S1mod2_S1_", "Link": "https://doi.org/10.1038/s41590-022-01230-1", "Author": "Phad et al., 2022", "Species": "human", "Age": "no", "BSource": "PBMC", "BType": "Plasma-B-Cells", "Vaccine": "None", "Disease": "None", "Subject": "Donor-2", "Longitudinal": "May-Year-2020", "Unique sequences": 2287, "Isotype": "All", "Chain": "Paired"}']

=== Shape ===
(2288, 1)

=== First 5 rows ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [3]:
df.columns

Index(['{"Run": "SRR28422122", "Link": "https://pubmed.ncbi.nlm.nih.gov/39548093/", "Author": "Dieudonne et al.", "Species": "human", "Age": 54.0, "BSource": "PBMC", "BType": "Unsorted-B-Cells", "Subject": "no", "Disease": "Primary Antiphospholipid Syndrome", "Vaccine": "None", "Longitudinal": "no", "Unique sequences": 816, "Isotype": "All", "Chain": "Paired"}'], dtype='object')

In [6]:
df["sequence_alignment_aa_heavy"]


KeyError: 'sequence_alignment_aa_heavy'

In [7]:
print(type(df.columns))
print(df.columns[:5])

<class 'pandas.core.indexes.base.Index'>
Index(['{"Run": "SRR28422122", "Link": "https://pubmed.ncbi.nlm.nih.gov/39548093/", "Author": "Dieudonne et al.", "Species": "human", "Age": 54.0, "BSource": "PBMC", "BType": "Unsorted-B-Cells", "Subject": "no", "Disease": "Primary Antiphospholipid Syndrome", "Vaccine": "None", "Longitudinal": "no", "Unique sequences": 816, "Isotype": "All", "Chain": "Paired"}'], dtype='object')


In [9]:
df.iloc[0]

{"Run": "SRR28422122", "Link": "https://pubmed.ncbi.nlm.nih.gov/39548093/", "Author": "Dieudonne et al.", "Species": "human", "Age": 54.0, "BSource": "PBMC", "BType": "Unsorted-B-Cells", "Subject": "no", "Disease": "Primary Antiphospholipid Syndrome", "Vaccine": "None", "Longitudinal": "no", "Unique sequences": 816, "Isotype": "All", "Chain": "Paired"}    ANARCI_status_light
Name: (sequence_id_heavy, sequence_heavy, locus_heavy, stop_codon_heavy, vj_in_frame_heavy, v_frameshift_heavy, productive_heavy, rev_comp_heavy, complete_vdj_heavy, v_call_heavy, d_call_heavy, j_call_heavy, sequence_alignment_heavy, germline_alignment_heavy, sequence_alignment_aa_heavy, germline_alignment_aa_heavy, v_alignment_start_heavy, v_alignment_end_heavy, d_alignment_start_heavy, d_alignment_end_heavy, j_alignment_start_heavy, j_alignment_end_heavy, v_sequence_alignment_heavy, v_sequence_alignment_aa_heavy, v_germline_alignment_heavy, v_germline_alignment_aa_heavy, d_sequence_alignment_heavy, d_sequence_ali

In [12]:
df = pd.read_csv(file_path, compression="gzip", header=0)
print(df.shape)
print(df.columns[:20])

df.iloc[0]

(817, 1)
Index(['{"Run": "SRR28422122", "Link": "https://pubmed.ncbi.nlm.nih.gov/39548093/", "Author": "Dieudonne et al.", "Species": "human", "Age": 54.0, "BSource": "PBMC", "BType": "Unsorted-B-Cells", "Subject": "no", "Disease": "Primary Antiphospholipid Syndrome", "Vaccine": "None", "Longitudinal": "no", "Unique sequences": 816, "Isotype": "All", "Chain": "Paired"}'], dtype='object')


{"Run": "SRR28422122", "Link": "https://pubmed.ncbi.nlm.nih.gov/39548093/", "Author": "Dieudonne et al.", "Species": "human", "Age": 54.0, "BSource": "PBMC", "BType": "Unsorted-B-Cells", "Subject": "no", "Disease": "Primary Antiphospholipid Syndrome", "Vaccine": "None", "Longitudinal": "no", "Unique sequences": 816, "Isotype": "All", "Chain": "Paired"}    ANARCI_status_light
Name: (sequence_id_heavy, sequence_heavy, locus_heavy, stop_codon_heavy, vj_in_frame_heavy, v_frameshift_heavy, productive_heavy, rev_comp_heavy, complete_vdj_heavy, v_call_heavy, d_call_heavy, j_call_heavy, sequence_alignment_heavy, germline_alignment_heavy, sequence_alignment_aa_heavy, germline_alignment_aa_heavy, v_alignment_start_heavy, v_alignment_end_heavy, d_alignment_start_heavy, d_alignment_end_heavy, j_alignment_start_heavy, j_alignment_end_heavy, v_sequence_alignment_heavy, v_sequence_alignment_aa_heavy, v_germline_alignment_heavy, v_germline_alignment_aa_heavy, d_sequence_alignment_heavy, d_sequence_ali

In [1]:
import pandas as pd
import json

file_path = "/home/ec2-user/SageMaker/InterPLM/oas_data/SRR28422122_1_Paired_All.csv.gz"

# --- Step 1: grab the first row (metadata) ---

# --- Step 2: load the rest as normal dataframe ---
df = pd.read_csv(file_path, compression="gzip", header=0, skiprows=1)

print("\nDataFrame shape:", df.shape)
print("First few columns:", df.columns[:10])
print("First row:")
print(df.iloc[0][["sequence_alignment_aa_heavy", "cdr3_aa_heavy", "sequence_alignment_aa_light", "cdr3_aa_light"]])



DataFrame shape: (816, 198)
First few columns: Index(['sequence_id_heavy', 'sequence_heavy', 'locus_heavy',
       'stop_codon_heavy', 'vj_in_frame_heavy', 'v_frameshift_heavy',
       'productive_heavy', 'rev_comp_heavy', 'complete_vdj_heavy',
       'v_call_heavy'],
      dtype='object')
First row:
sequence_alignment_aa_heavy    QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKG...
cdr3_aa_heavy                                                     ARGAAAAGTLPLDY
sequence_alignment_aa_light    EIVLTQSPATLSLSPGERATLSCRASQSVSSYLAWYQQKPGQAPRL...
cdr3_aa_light                                                        QQRSNWPPSLT
Name: 0, dtype: object


In [4]:
df.columns

Index(['sequence_id_heavy', 'sequence_heavy', 'locus_heavy',
       'stop_codon_heavy', 'vj_in_frame_heavy', 'v_frameshift_heavy',
       'productive_heavy', 'rev_comp_heavy', 'complete_vdj_heavy',
       'v_call_heavy',
       ...
       'cdr3_end_light', 'np1_light', 'np1_length_light', 'np2_light',
       'np2_length_light', 'c_region_light', 'Isotype_light',
       'Redundancy_light', 'ANARCI_numbering_light', 'ANARCI_status_light'],
      dtype='object', length=198)

In [5]:
with open("columns.txt", "w") as f:
    for col in df.columns:
        f.write(f"{col}\n")

In [2]:
len(df.columns)

198

In [3]:
print(df.iloc[0][["sequence_alignment_aa_heavy"]])


sequence_alignment_aa_heavy    QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKG...
Name: 0, dtype: object


In [14]:
import pandas as pd
import json, gzip
from pathlib import Path

file_path = Path("/home/ec2-user/SageMaker/InterPLM/oas_data/SRR28422122_1_Paired_All.csv.gz")

# ---- Step 1: read the first line as JSON metadata ----
with gzip.open(file_path, "rt") as fh:
    first_line = fh.readline().strip()
metadata = json.loads(first_line)
print("✅ Parsed metadata keys:", list(metadata.keys()))
print("Run:", metadata.get("Run"), "| Author:", metadata.get("Author"))

# ---- Step 2: load the table (most OAS dumps are TSV) ----
def load_oas_table(fp: Path):
    # Try TSV first (most common)
    df = pd.read_csv(fp, compression="gzip", sep="\t", skiprows=1, dtype=str)
    if df.shape[1] == 1:  # still one big column? fall back to Python engine sniffing
        df = pd.read_csv(fp, compression="gzip", sep=None, engine="python", skiprows=1, dtype=str)
    # If somehow it's still one wide column, try to split by large whitespace
    if df.shape[1] == 1:
        only_col = df.columns[0]
        split = df[only_col].str.split(r"\s{2,}|\t", regex=True, expand=True)
        # assume first row of split contains headers
        split.columns = split.iloc[0]
        df = split.iloc[1:].reset_index(drop=True)
    return df

df = load_oas_table(file_path)

print("\n📐 DataFrame shape:", df.shape)
print("🧭 First 10 columns:", list(df.columns[:10]))

# ---- Step 3: quick peek at the sequence AA columns if present ----
wanted_cols = [
    "sequence_alignment_aa_heavy",
    "cdr3_aa_heavy",
    "sequence_alignment_aa_light",
    "cdr3_aa_light",
]
present = [c for c in wanted_cols if c in df.columns]
if present:
    display(df[present].head())
else:
    # help you discover the right names if they differ slightly
    like_heavy = [c for c in df.columns if "aa" in c and "heavy" in c]
    like_light = [c for c in df.columns if "aa" in c and "light" in c]
    print("\nColumns containing 'aa' & 'heavy':", like_heavy[:20])
    print("Columns containing 'aa' & 'light':", like_light[:20])

# Example: access the heavy AA alignment column
# df["sequence_alignment_aa_heavy"].head()


JSONDecodeError: Extra data: line 1 column 4 (char 3)

In [6]:
def build_variable_domain(df, prefix="heavy"):
    parts = [
        f"fwr1_aa_{prefix}",
        f"cdr1_aa_{prefix}",
        f"fwr2_aa_{prefix}",
        f"cdr2_aa_{prefix}",
        f"fwr3_aa_{prefix}",
        f"cdr3_aa_{prefix}",
        f"fwr4_aa_{prefix}",
    ]
    return df[parts].fillna("").agg("".join, axis=1)

df["VH_clean"] = build_variable_domain(df, "heavy")
df["VL_clean"] = build_variable_domain(df, "light")

In [8]:
df.iloc[0]["sequence_alignment_aa_heavy"]

'QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARGAAAAGTLPLDYWGQGTLVTVSS'

In [7]:
df.iloc[0]["VH_clean"]

'QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARGAAAAGTLPLDYWGQGTLVTVSS'

In [10]:
def print_antibody_segments(df, row_idx=0):
    """Pretty-print FWR/CDR segments for heavy and light chains of one antibody."""

    def show_chain(prefix, label):
        print(f"\n=== {label} Chain ({prefix}) ===")
        for seg in ["fwr1", "cdr1", "fwr2", "cdr2", "fwr3", "cdr3", "fwr4"]:
            col = f"{seg}_aa_{prefix}"
            if col in df.columns:
                seq = df.at[row_idx, col]
                print(f"{seg.upper():<5}: {seq if pd.notna(seq) else ''}")

    show_chain("heavy", "Heavy (VH)")
    show_chain("light", "Light (VL)")


# Example usage: look at the first row
print_antibody_segments(df, row_idx=0)



=== Heavy (VH) Chain (heavy) ===
FWR1 : QLQLQESGPGLVKPSETLSLTCTVS
CDR1 : GGSISSSSYY
FWR2 : WGWIRQPPGKGLEWIGS
CDR2 : IYYSGST
FWR3 : YYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYC
CDR3 : ARGAAAAGTLPLDY
FWR4 : WGQGTLVTVSS

=== Light (VL) Chain (light) ===
FWR1 : EIVLTQSPATLSLSPGERATLSCRAS
CDR1 : QSVSSY
FWR2 : LAWYQQKPGQAPRLLIY
CDR2 : DAS
FWR3 : NRATGIPARFSGSGSGTDFTLTISSLEPEDFAVYYC
CDR3 : QQRSNWPPSLT
FWR4 : FGGGTKVEIK
